In [1]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from jaxtyping import Float, Int, Bool
from torch import Tensor
from tqdm.auto import tqdm
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
import plotly.express as px 
from collections import defaultdict
import matplotlib.pyplot as plt
import re
from IPython.display import display, HTML
from datasets import load_dataset
from collections import Counter
import pickle
import os
import haystack_utils
from transformer_lens import utils
from fancy_einsum import einsum
import einops
import json
import ipywidgets as widgets
from IPython.display import display
from datasets import load_dataset
import random
import math
import random
import neel.utils as nutils
from neel_plotly import *

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

%reload_ext autoreload
%autoreload 2

In [2]:
model: HookedTransformer = HookedTransformer.from_pretrained("gpt2-large")

data = load_dataset("stas/openwebtext-10k", split="train")
strings = [i for i in data["text"] if len(i)>2000]
len(strings)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-large into HookedTransformer


Repo card metadata block was not found. Setting CardData to empty.


7083

In [3]:
num_prompts = 64
n_ctx = 256
tokens = model.to_tokens(strings[num_prompts:2*num_prompts])[:, :n_ctx]
tokens.shape

logits = model(tokens)
A = model.to_single_token(" a")
AN = model.to_single_token(" an")
logit_diff = logits[:, :, AN] - logits[:, :, A]
# -5 where token is "an", diff otherwise
logit_diff = torch.where(tokens==AN, torch.zeros_like(logit_diff)-5, logit_diff)

In [4]:
token_df = pd.DataFrame({"logit_diff": logit_diff.flatten().cpu().numpy(), "batch": [b for b in range(num_prompts) for p in range(n_ctx)], "pos": [p for b in range(num_prompts) for p in range(n_ctx)]})
nutils.show_df(token_df.sort_values("logit_diff", ascending=False).head(100))

,logit_diff,batch,pos
436,7.321955,1,180
12375,3.947590,48,87
12373,3.835551,48,85
12391,3.288544,48,103
12439,2.976463,48,151
12389,2.804193,48,101
12437,2.747065,48,149
6655,2.399273,25,255
16157,2.126307,63,29
12666,1.691870,49,122


In [5]:
temp_df = token_df.sort_values("logit_diff", ascending=False).head(20)
import circuitsvis as cv
for row in temp_df.iterrows():
    b = int(row[1]["batch"])
    p = int(row[1]["pos"])
    #print(b, p)
    str_toks = model.to_str_tokens(tokens[b, :p+3])
    vals = logit_diff[b, :p+3]
    haystack_utils.print_strings_as_html(str_toks, vals.tolist(), max_value=5)
    #nutils.create_html(str_toks, vals)
    #display(cv.tokens.colored_tokens(str_toks, vals))


In [ ]:
utils.test_prompt("Ross Perot, who ran as", "an", model)
utils.test_prompt("Van der Bellen, who ran as", "an", model)
utils.test_prompt("Senator Angus King, who ran as", "an", model)
utils.test_prompt("Spot gold was up 0.7 percent at $1,340.90", "an", model)
utils.test_prompt("An eye for", "an", model)
utils.test_prompt("Every action has", "an", model)
utils.test_prompt("I'll be there in under half", "an", model)
utils.test_prompt("I climbed up the pear tree and picked a pear. I climbed up the apple tree and picked", "an", model)

In [31]:
prompts = [
    "Elephants are large animals. The only animal larger than",
    "Ostriches are large animals. The only animals larger than",
    "Alligators are large animals. The only animals larger than",
    "Emus are large animals. The only animals larger than",
    "Bears are large animals. The only animal larger than",
    "Tigers are large animals. The only animal larger than",
    "Whales are large animals. The only animal larger than",
    "Giraffes are large animals. The only animal larger than",
]

animals = ["Elephants", "Ostriches", "Alligators", "Emus", "Bears", "Tigers", "Whales", "Giraffes"]

for prompt in prompts:
    utils.test_prompt(prompt, "an", model)

Tokenized prompt: ['<|endoftext|>', 'Ele', 'ph', 'ants', ' are', ' large', ' animals', '.', ' The', ' only', ' animal', ' larger', ' than']
Tokenized answer: [' an']


Performance on answer token:
Rank: 0        Logit: 17.89 Prob: 41.44% Token: | an|

Top 0th token. Logit: 17.89 Prob: 41.44% Token: | an|
Top 1th token. Logit: 17.17 Prob: 20.22% Token: | a|
Top 2th token. Logit: 16.68 Prob: 12.44% Token: | them|
Top 3th token. Logit: 16.44 Prob:  9.78% Token: | elephants|
Top 4th token. Logit: 15.73 Prob:  4.81% Token: | the|
Top 5th token. Logit: 14.37 Prob:  1.24% Token: | elephant|
Top 6th token. Logit: 14.25 Prob:  1.09% Token: | their|
Top 7th token. Logit: 13.67 Prob:  0.61% Token: | these|
Top 8th token. Logit: 13.65 Prob:  0.60% Token: | humans|
Top 9th token. Logit: 13.48 Prob:  0.50% Token: | they|


Ranks of the answer tokens: [(' an', 0)]

Tokenized prompt: ['<|endoftext|>', 'O', 'st', 'ric', 'hes', ' are', ' large', ' animals', '.', ' The', ' only', ' animals', ' larger', ' than']
Tokenized answer: [' an']


...

Ranks of the answer tokens: [(' an', 2)]

Tokenized prompt: ['<|endoftext|>', 'G', 'ir', 'aff', 'es', ' are', ' large', ' animals', '.', ' The', ' only', ' animal', ' larger', ' than']
Tokenized answer: [' an']


Performance on answer token:
Rank: 4        Logit: 16.09 Prob:  3.90% Token: | an|

Top 0th token. Logit: 18.57 Prob: 46.52% Token: | a|
Top 1th token. Logit: 18.09 Prob: 28.88% Token: | gir|
Top 2th token. Logit: 16.93 Prob:  9.03% Token: | them|
Top 3th token. Logit: 16.11 Prob:  3.96% Token: | the|
Top 4th token. Logit: 16.09 Prob:  3.90% Token: | an|
Top 5th token. Logit: 14.72 Prob:  0.99% Token: | their|
Top 6th token. Logit: 14.71 Prob:  0.98% Token: | Gir|
Top 7th token. Logit: 14.15 Prob:  0.56% Token: | g|
Top 8th token. Logit: 14.07 Prob:  0.52% Token: | one|
Top 9th token. Logit: 13.93 Prob:  0.45% Token: | these|


Ranks of the answer tokens: [(' an', 4)]

In [30]:
# prompts = [
# "Ross Perot, who ran as",
# "Van der Bellen, who ran as",
# "Senator Angus King, who ran as",
# "Spot gold was up 0.7 percent at $1,340.90",
# "An eye for",
# "Every action has",
# "I'll be there in under half",
# "I climbed up the pear tree and picked a pear. I climbed up the apple tree and picked",
# "I climbed up the apple tree and picked an apple. I climbed up the pear tree and picked",
# ]

# prompts = [
#     "I climbed up the pear tree and picked a pear. I climbed up the apple tree and picked",
#     "I climbed up the pear tree and picked a pear. I climbed up the orange tree and picked",
#     "I climbed up the apple tree and picked an apple. I climbed up the pear tree and picked",
#     "I climbed up the apple tree and picked an apple. I climbed up the banana tree and picked",
#     "I climbed up the apple tree and picked an apple. I climbed up the cherry tree and picked",
#     "I climbed up the apple tree and picked an apple. I climbed up the peach tree and picked",
#     "I climbed up the apple tree and picked an apple. I climbed up the plum tree and picked",
# ]

fruits = ["apple", "orange", "pear", "banana", "cherry", "peach", "plum"]

tokens = model.to_tokens(prompts)
final_index = (tokens!=model.tokenizer.pad_token_id).sum(-1)
tokens[np.arange(len(prompts)), final_index]
# tokens[np.arange(len(prompts)), final_index+1]

tensor([621, 621, 621, 621, 621, 621, 621, 621], device='cuda:0')

In [18]:
for prompt in prompts:
    utils.test_prompt(prompt, "an", model)

Tokenized prompt: ['<|endoftext|>', 'Ross', ' Per', 'ot', ',', ' who', ' ran', ' as']
Tokenized answer: [' an']


Performance on answer token:
Rank: 0        Logit: 19.55 Prob: 40.33% Token: | an|

Top 0th token. Logit: 19.55 Prob: 40.33% Token: | an|
Top 1th token. Logit: 19.40 Prob: 34.84% Token: | a|
Top 2th token. Logit: 18.92 Prob: 21.50% Token: | the|
Top 3th token. Logit: 14.48 Prob:  0.25% Token: | "|
Top 4th token. Logit: 14.37 Prob:  0.23% Token: | independent|
Top 5th token. Logit: 14.29 Prob:  0.21% Token: | one|
Top 6th token. Logit: 14.24 Prob:  0.20% Token: | Republican|
Top 7th token. Logit: 13.91 Prob:  0.14% Token: | part|
Top 8th token. Logit: 13.74 Prob:  0.12% Token: | president|
Top 9th token. Logit: 13.63 Prob:  0.11% Token: | governor|


Ranks of the answer tokens: [(' an', 0)]

Tokenized prompt: ['<|endoftext|>', 'Van', ' der', ' Bell', 'en', ',', ' who', ' ran', ' as']
Tokenized answer: [' an']


...

Ranks of the answer tokens: [(' an', 0)]

Tokenized prompt: ['<|endoftext|>', 'I', ' climbed', ' up', ' the', ' apple', ' tree', ' and', ' picked', ' an', ' apple', '.', ' I', ' climbed', ' up', ' the', ' pear', ' tree', ' and', ' picked']
Tokenized answer: [' an']


Performance on answer token:
Rank: 0        Logit: 20.08 Prob: 55.05% Token: | an|

Top 0th token. Logit: 20.08 Prob: 55.05% Token: | an|
Top 1th token. Logit: 19.54 Prob: 32.06% Token: | a|
Top 2th token. Logit: 17.37 Prob:  3.65% Token: | another|
Top 3th token. Logit: 16.97 Prob:  2.43% Token: | two|
Top 4th token. Logit: 16.78 Prob:  2.02% Token: | some|
Top 5th token. Logit: 15.67 Prob:  0.67% Token: | up|
Top 6th token. Logit: 15.34 Prob:  0.48% Token: | three|
Top 7th token. Logit: 15.05 Prob:  0.36% Token: | pear|
Top 8th token. Logit: 15.02 Prob:  0.35% Token: | the|
Top 9th token. Logit: 14.63 Prob:  0.24% Token: | one|


Ranks of the answer tokens: [(' an', 0)]

In [32]:
unembed_dir = model.W_U[:, AN] - model.W_U[:, A]
logits, cache = model.run_with_cache(model.to_tokens(prompts))
resid_stack, resid_labels = cache.get_full_resid_decomposition(expand_neurons=False, apply_ln=True, return_labels=True)
final_resid_stack = resid_stack[:, np.arange(len(prompts)), final_index, :]
(final_resid_stack @ unembed_dir).shape

Tried to stack head results when they weren't cached. Computing head results now


torch.Size([759, 8])

In [33]:
line((final_resid_stack @ unembed_dir).T, line_labels=[a for a in animals], x=resid_labels, title="Direct Logit Diff Attribution")
#line((final_resid_stack @ unembed_dir).T, line_labels=[p[:10] for p in prompts], x=resid_labels, title="Direct Logit Diff Attribution")
#line((final_resid_stack @ unembed_dir).T, line_labels=[f for f in fruits], x=resid_labels, title="Direct Logit Diff Attribution")


In [34]:
final_ln_scale = cache["scale"][np.arange(len(prompts)), final_index, 0]
print(final_ln_scale.shape)
for layer in [31, 34, 35]:
    neuron_acts = cache["post", layer][np.arange(len(prompts)), final_index, :]
    neuron_wdla = (model.blocks[layer].mlp.W_out @ unembed_dir) / final_ln_scale[:, None]
    #line(neuron_acts * neuron_wdla, line_labels=[f for f in fruits], title=f"Neuron DLA in L{layer}")
    line(neuron_acts * neuron_wdla, line_labels=[a for a in animals], title=f"Neuron DLA in L{layer}")
    # line(neuron_acts * neuron_wdla, line_labels=[p[:10] for p in prompts], title=f"Neuron DLA in L{layer}")

torch.Size([8])


In [36]:
alg_prompts = ([
    "I climbed up the pear tree and picked a pear. I climbed up the apple tree and picked",
    "I climbed up the pear tree and picked a pear. I climbed up the orange tree and picked",
    "I climbed up the pear tree and picked a pear. I climbed up the elephant tree and picked",
    "I climbed up the pear tree and picked a pear. I climbed up the pineapple tree and picked",
    "I climbed up the pear tree and picked a pear. I climbed up the strawberry tree and picked",
    "I climbed up the pear tree and picked a pear. I climbed up the plum tree and picked",
])
token_list = [f"{s}/{i}" for i, s in enumerate(model.to_str_tokens(alg_prompts[-1]))]
alg_tokens = model.to_tokens(alg_prompts)
alg_logits, alg_cache = model.run_with_cache(alg_tokens)
# line(alg_logits[:, -1, AN] - alg_logits[:, -1, A])
# %%
imshow(alg_cache["pattern", 26][:, 1, :, :], facet_col=0, x=token_list, y=token_list)

In [37]:
common_objects = [
    "pen", "hat", "cup", "bag", "box", "car", "dog", "cat", "key", "bed",
    "pot", "pan", "jar", "jug", "rug", "bat", "ball", "shoe", "ship", "bike",
    "desk", "door", "bell", "book", "bowl", "coin", "comb", "cord", "disk", "doll",
    "drum", "flag", "fork", "lamp", "lock", "mug", "nail", "pipe", "ring", "rope",
    "seed", "skirt", "spoon", "stamp", "star", "stick", "tent", "tie", "tooth", "toy",
    "tree", "watch", "whip", "bird", "boat", "boot", "cane", "card", "chain", "chair",
    "chalk", "clock", "cloth", "cloud", "coat", "crab", "disk", "dress", "drop", "drum",
    "duck", "dust", "fence", "flag", "floor", "flower", "fly", "fog", "fork", "fruit",
    "glass", "glove", "grass", "hair", "hand", "harp", "hat", "hill", "horn", "horse",
    "house", "island", "jewel", "jug", "kettle", "key", "kite", "knife", "leaf", "leg",
    "library", "light", "line", "loaf", "lock", "machine", "man", "map", "moon", "net",
    "nose", "nut", "office", "orange", "oven", "parcel", "pen", "pencil", "picture", "pig",
    "pin", "pipe", "plane", "plate", "plough", "pocket", "pot", "potato", "prison", "pump",
    "rail", "rat", "receipt", "ring", "rod", "roof", "root", "sail", "school", "scissors",
    "screw", "seed", "sheep", "shelf", "ship", "shirt", "shoe", "skin", "skirt", "snake",
    "sock", "spade", "sponge", "spoon", "spring", "square", "stamp", "star", "station", "stem",
    "stick", "stocking", "stomach", "store", "street", "sun", "table", "tail", "thread", "throat",
    "thumb", "ticket", "toe", "tongue", "tooth", "town", "train", "tray", "tree", "trousers",
    "umbrella", "wall", "watch", "wheel", "whistle", "window", "wire", "wing", "worm", "yarn"
]
vowel_objects = [
    "apple", "apron", "arm", "ankle", "arrow", "atom", "ant", "anchor", "album", "axe",
    "ear", "egg", "elbow", "engine", "eagle", "earring", "envelope", "eye", "eel", "earth",
    "ice", "iron", "ink", "island", "ivy", "igloo", "insect", "instrument", "image", "indicator",
    "oak", "oar", "ocean", "octopus", "onion", "orange", "organ", "oven", "owl", "ox",
    "umbrella", "urn", "utensil", "uniform", "ukelele", "unit", "unicorn", "upstairs", "underwear", "urchin",
    "emerald", "end", "elephant", "elm", "easel", "eraser", "eskimo", "entrance", "estate", "echo",
    "ash", "art", "armchair", "air", "arch", "anvil", "alloy", "alley", "atom", "amulet",
    "olive", "opera", "opal", "ottoman", "orchid", "orbit", "ostrich", "oxen", "oil", "ounce",
    "iceberg", "iris", "idea", "iguanodon", "inlet", "icon", "input", "isle", "itch", "issue",
    "udder", "uplift", "update", "upgrade", "undo", "uptake", "upbeat", "upturn", "upload", "upstream",
    "antenna", "almond", "arena", "aorta", "ape", "asteroid", "aster", "auction", "audio", "avocado",
    "edge", "eel", "eel", "equipment", "escalator", "essence", "emblem", "echo", "engineer", "equator",
    "opal", "orchard", "oboe", "oval", "oven", "overcoat", "oyster", "ounce", "outlet", "outline",
    "aerial", "airplane", "awning", "award", "agent", "agate", "arc", "arena", "armadillo", "apricot"
]
word_list = common_objects[:50] + vowel_objects[:50]
token_lengths = [len(model.to_tokens(" "+word, prepend_bos=False).squeeze(0)) for word in word_list]
word_list = [word_list[i] for i in range(len(word_list)) if token_lengths[i]==1]
len(word_list)

89

In [38]:
prompt_template = "I climbed up the pear tree and picked a pear. I climbed up the {} tree and picked"
prompt_list = [prompt_template.format(word) for word in word_list]
tree_tokens = model.to_tokens(prompt_list)
model.to_str_tokens(tree_tokens[0])
# %%
tree_logits, tree_cache = model.run_with_cache(tree_tokens, names_filter=lambda name: "pattern" in name or "post" in name or "z" in name or "out" in name or "scale" in name)
tree_neuron_acts = tree_cache['post', 31][:, -1, 892]
tree_attn = tree_cache['pattern', 26][:, 1, -1, 16]
tree_logit_diff = tree_logits[:, -1, AN] - tree_logits[:, -1, A]
tree_df = pd.DataFrame(dict(
    word=word_list,
    neuron_acts=to_numpy(tree_neuron_acts),
    attn=to_numpy(tree_attn),
    logit_diff=to_numpy(tree_logit_diff),
    is_vowel = [word[0].lower() in "aeiou" for word in word_list],
))
nutils.focus_df_column(tree_df, "logit_diff")

,word,neuron_acts,attn,logit_diff,is_vowel
50,apple,5.442508,0.965466,0.986042,True
60,egg,5.412023,0.866979,0.621611,True
82,ox,4.548833,0.880863,0.243952,True
63,eagle,4.275532,0.642528,0.164066,True
81,owl,4.736547,0.869019,0.121283,True
58,axe,4.502692,0.819802,0.120953,True
78,orange,4.382184,0.876488,0.108707,True
55,ant,4.668735,0.876618,0.107754,True
77,onion,4.587779,0.959486,0.062597,True
53,arrow,4.423998,0.456668,0.011061,True
